# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링

- user_profile_df : User의 아이디, 리뷰수, 팔로워, url을 저장함
- user_df : '아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수' 를 저장함


### 문제

1. 인기 리뷰어라면 핫플레이스만 다녀서 평범한 음식점에 대한 정보는 못얻지 않을까?
2. 리뷰어는 어디서 가져오지?
3. 연도를 어떻게 하지 ? 연도가 없는 경우도 있고, 연도와 요일이 없는 경우도 있음 

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  git pull origin main

from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import os

import pandas as pd
from selenium.webdriver.common.by import By

c:\Users\hare0\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


### 1. User 정보를 가져오기 위해 (현재 기준 없음) user의 myplace url을 복사한다. 

In [2]:
url = 'https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2'


User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 

In [3]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()

# User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 
def user_profile(url) :
    # 주소 이동
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    user_element = {}

    # User ID 찾기
    user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
    user_element['아이디'] = user_id_element.text if user_id_element else None

    # User의 인기도 : 리뷰수, 팔로워 찾기 
    user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
    
    for element in user_popularity:
        em_tag = element.find('em')
        if em_tag:
            key = element.text.replace(em_tag.text, '').strip()
            value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
            user_element[key] = value
    
    # User URL
    user_element['주소'] = url

    print(user_element)

    return user_element


### 유저 정보 찾기

In [4]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()

# User의 아이디, 리뷰수, 팔로워를 가져오는 함수 
user_data = user_profile(url)


# User 정보 저장 
user_profile_df = pd.DataFrame([user_data], index=[0])
user_profile_df

{'아이디': 'xll****', '리뷰': 983, '팔로잉': 0, '팔로워': 75, '주소': 'https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2'}


,아이디,리뷰,팔로잉,팔로워,주소
0,xll****,983,0,75,https://m.place.naver.com/my/5c36b9f1e511a8856...


In [5]:
# 맨 처음 게시물 클릭 
button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
button.click()
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')

# data를 담을 빈 리스트 선언
data_list = [] 


### 매장 정보 찾기

In [81]:
# 페이지 아래로 스크롤 몇번 
page_down = 1

# 페이지 스크롤 횟수만큼 반복
for _ in range(page_down):

    # 대기시간 5초 
    wait = WebDriverWait(driver, 5)

    # 요소를 찾을 때 대기 조건 추가
    user_review_elements = wait.until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92'))
    )

    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 

    # 리뷰에서 정보 가져오기
    for reviews_elements in user_review_elements:
        i = 0 # 카테고리와 주소 구분할 때 사용
        restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
        
        if bool(restaurant_elements): # 정보가 없는 경우가 있음

            # 매장명 찾기
            restaurant_name = restaurant_elements[0].text  

    #<--------------------------------------------------------------------------------------------------->
            # 카테고리와 주소 찾기 
            category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
            category_location_soup = BeautifulSoup(category_location_elements[i].get_attribute('outerHTML'), 'html.parser')
            
            # span 태그 안에 있는 텍스트 가져오기
            span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
            category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
            location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
            i += 1
    #<--------------------------------------------------------------------------------------------------->
            # 리뷰 찾기 
            review_text = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')[0].text # 리뷰 본문
            sub_info=reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') #세부정보
            if bool(sub_info):
                sub_info=sub_info[0].text
            else:
                sub_info=None

    #<--------------------------------------------------------------------------------------------------->
            # 태그 찾기 
            # 일정 개수가 넘어가면 리뷰가 숨겨져 표시되므로 
            reactions_elements=reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트
            command=False
            for x in reactions_elements:
                if x.get_attribute("role"): # 리액션 숨김 없애기
                    x.click()
                    command=True
                else:
                    continue
            if command:
                reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트 재탐색
            reactions=list(map(lambda x:x.text,reactions_elements))

    #<--------------------------------------------------------------------------------------------------->
            # 방문 일자, 재방문 이력 찾기
            date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
            date=[x.text for x in date_info]

            if date:
                day = date[0].split('\n')[0] 
                revisit = date[1].replace('번째 방문', '')
        
            # 정보 추가
            data_dict = {
                    '아이디' : user_data['아이디'],
                    '리뷰' : user_data['리뷰'],
                    '팔로워' : user_data['팔로워'],
                    '매장명': restaurant_name,
                    '카테고리': category,
                    '주소' : location,
                    '리뷰 내용': review_text,
                    "세부정보" : sub_info,
                    '태그': reactions,
                    '방문일자': day,
                    '재방문횟수':revisit}
            data_list.append(data_dict)
    user_df = pd.DataFrame(data_list)
        
user_df

ElementClickInterceptedException: Message: element click intercepted: Element <span role="button" class="COw42b" tabindex="0">...</span> is not clickable at point (316, 13). Other element would receive the click: <button class="wTaI4v TFuZU7" type="button">...</button>
  (Session info: chrome=121.0.6167.161)


# 위 내용을 반복문으로 한번에 하기 

In [84]:
user_df = pd.DataFrame()

# 새로운 컬럼 추가
new_columns = ['아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수']

for column in new_columns:
    user_df[column] = None  # 또는 원하는 초기값으로 설정

user_df

,아이디,리뷰,팔로워,매장명,카테고리,주소,리뷰 내용,세부정보,태그,방문일자,재방문횟수


In [69]:
# myplace url
user_list = ['https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2',
'https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2',
# 'https://m.place.naver.com/my/5bd2f3682db9d1bdd38eedca/review?v=2', # 맨 처음 테마가 있으니까 에러남 
'https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2',
'https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2',
'https://m.place.naver.com/my/6010e880e71246c530be8c27/review?v=2',
'https://m.place.naver.com/my/5e27e4ec8f87a842bcb3505c/review?v=2']

In [92]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()


for url in user_list : 

    try : 
        # User의 아이디, 리뷰수, 팔로워를 가져오는 함수 
        user_data = user_profile(url)
        user_data_df = pd.DataFrame([user_data], index=[0])

        # 만약 User의 DataFrame이 이미 존재한다면, 기존 DataFrame에 추가
        if user_profile_df.index[-1] > 0:
            user_profile_df = pd.concat([user_profile_df, user_data_df], ignore_index=True)
            user_profile_df = user_profile_df.drop_duplicates() # 중복제거
        else:
            user_profile_df = user_data_df

        # 맨 처음 게시물 클릭 
        button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
        button.click()
        time.sleep(5)

        # # 반복해서 스크롤하기
        # n = 0 

        # while n < 5 :

        #     last_scroll_position = driver.execute_script("return window.scrollY")
        #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #     time.sleep(2)
        #     current_scroll_position = driver.execute_script("return window.scrollY")
            
        #     # 스크롤 위치에 변화가 없으면 종료
        #     if current_scroll_position == last_scroll_position:
        #         break
            
        #     # 현재 스크롤 위치를 다음 비교를 위해 저장
        #     last_scroll_position = current_scroll_position

        #     n += 1



        # data를 담을 빈 리스트 선언
        data_list = [] 

        # 페이지 아래로 스크롤 몇번
        page_down = 10

        # 페이지 스크롤 횟수만큼 반복
        for _ in range(page_down):

            # 대기시간 5초 
            wait = WebDriverWait(driver, 5)

            # 요소를 찾을 때 대기 조건 추가
            user_review_elements = wait.until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92'))
            )

            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 

            html = driver.page_source
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 리뷰에서 정보 가져오기
            for reviews_elements in user_review_elements:
                i = 0 # 카테고리와 주소 구분할 때 사용
                restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
                
                if bool(restaurant_elements): # 정보가 없는 경우가 있음

                    # 매장명 찾기
                    restaurant_name = restaurant_elements[0].text  

            #<--------------------------------------------------------------------------------------------------->
                    # 카테고리와 주소 찾기 
                    category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
                    category_location_soup = BeautifulSoup(category_location_elements[i].get_attribute('outerHTML'), 'html.parser')
                    
                    # span 태그 안에 있는 텍스트 가져오기
                    span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
                    category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
                    location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
                    i += 1
            #<--------------------------------------------------------------------------------------------------->
                    # 리뷰 찾기 
                    review_text = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')[0].text # 리뷰 본문
                    sub_info=reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') #세부정보
                    if bool(sub_info):
                        sub_info=sub_info[0].text
                    else:
                        sub_info=None

            #<--------------------------------------------------------------------------------------------------->
                    # 태그 찾기 
                    # 일정 개수가 넘어가면 리뷰가 숨겨져 표시되므로 
                    reactions_elements=reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트
                    command=False
                    for x in reactions_elements:
                        if x.get_attribute("role"): # 리액션 숨김 없애기
                            x.click()
                            command=True
                        else:
                            continue
                    if command:
                        reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트 재탐색
                    reactions=list(map(lambda x:x.text,reactions_elements))
                    reactions = str(reactions).replace('[', '').replace(']', '')
                    


            #<--------------------------------------------------------------------------------------------------->
                    # 방문 일자, 재방문 이력 찾기
                    date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
                    date=[x.text for x in date_info]

                    if date:
                        day = date[0].split('\n')[0]
                        if day[-1:].isdigit(): # 요일이 없으면 
                            weekday = day[-1:]
                        else:
                            weekday = ""
                        day = day[:-2]
                        revisit = int(date[1].replace('번째 방문', ''))

                
                    # 정보 추가
                    data_dict = {
                            '아이디' : user_data['아이디'],
                            '리뷰' : user_data['리뷰'],
                            '팔로워' : user_data['팔로워'],
                            '매장명': restaurant_name,
                            '카테고리': category,
                            '주소' : location,
                            '리뷰 내용': review_text,
                            "세부정보" : sub_info,
                            '태그': reactions,
                            '방문일자': day,
                            '방문요일' : weekday,
                            '재방문횟수': revisit}
                    data_list.append(data_dict)

            if 'user_df' in locals():
                user_df = pd.concat([user_df, pd.DataFrame(data_list)], ignore_index=True)
            else:
                user_df = pd.DataFrame(data_list)

    except Exception as e:
        print(f"에러 메시지: {str(e)}")
        continue
                
user_df


{'아이디': 'xll****', '리뷰': 983, '팔로잉': 0, '팔로워': 75, '주소': 'https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2'}
에러 메시지: list index out of range
{'아이디': '맛있는거 먹으려고 운동함', '리뷰': 534, '팔로잉': 2, '팔로워': 41, '주소': 'https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2'}
{'아이디': '참참s', '리뷰': 495, '팔로잉': 0, '팔로워': 74, '주소': 'https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2'}
에러 메시지: Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3P-5HQ"}
  (Session info: chrome=121.0.6167.161)

{'아이디': 'JUDY0725', '리뷰': 1927, '팔로잉': 0, '팔로워': 95, '주소': 'https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2'}
{'아이디': '초코피스타치오', '리뷰': 159, '팔로잉': 6, '팔로워': 24, '주소': 'https://m.place.naver.com/my/6010e880e71246c530be8c27/review?v=2'}
{'아이디': 'sodadada7', '리뷰': 179, '팔로잉': 1, '팔로워': 0, '주소': 'https://m.place.naver.com/my/5e27e4ec8f87a842bcb3505c/review?v=2'}


,아이디,리뷰,팔로워,매장명,카테고리,주소,리뷰 내용,세부정보,태그,방문일자,재방문횟수,방문요일
0,xll****,983,75,매우매오 강남역별관,"육류,고기요리",서울특별시 강남구 역삼동,구성이 좋아요! 끓이면서 따끈하게 먹을 수 있어 좋아요~,이용 방법예약 없이대기 시간바로 입장동행친구,'메뉴 구성이 알차요',2.13,1,
1,맛있는거 먹으려고 운동함,534,41,메디큐브의원 강남신논현역,피부과,서울특별시 서초구 서초동,"강남역 피부과입니다! 신논현역, 강남역에서 도보로 10분 내로 방문할 수 있어서 접...",이용 방법예약 후대기 시간바로 입장,,2.5,3,
2,참참s,495,74,우몽블랙 판교점,"육류,고기요리",경기도 성남시 분당구 백현동,고급스러운 프라이빗한 룸에서 한우를 직접 맛있게 구워주셔서 너무 특별한 식사였어요....,이용 방법예약 후대기 시간바로 입장목적친목동행친척·형제,"'직접 잘 구워줘요', '음식이 맛있어요', '고기 질이 좋아요', '양이 많아요'...",2.11,1,
3,JUDY0725,1927,95,옵스 마린시티점,베이커리,부산광역시 해운대구 우동,붐비지 않아서 쾌적하게 빵을 고를 수 있었어요! \n친절하게 설명해주시고 선물세트가...,이용 방법예약 없이대기 시간10분 이내목적나들이동행친구,"'빵이 맛있어요', '가성비가 좋아요', '디저트가 맛있어요', '특별한 메뉴가 있...",2.12,1,
4,초코피스타치오,159,24,메종 아카이,돈가스,서울특별시 종로구 명륜2가,혜화 샤브샤브 맛집 메종아카이 일본풍 아기자기한 분위기와 깔끔하고 정갈한 플레이팅이...,None,"'음식이 맛있어요', '특별한 메뉴가 있어요', '인테리어가 멋져요', '단체모임 ...",2.10,1,
...,...,...,...,...,...,...,...,...,...,...,...,...
4791,sodadada7,179,0,오생,"오뎅,꼬치",대구광역시 달서구 성당동,살면서 처음본 하이볼 타워가 있었던 대구 서부 정류장 맛집이에요😉ㅋㅋㅋㅋ 여섯잔 분...,None,"'술이 다양해요', '특별한 메뉴가 있어요', '대화하기 좋아요', '친절해요'",23.7.7,1,
4792,sodadada7,179,0,복골상회,없음,없음,고기 다 구워 주시니까 너무 편리하고 맛있어요~~~~\n채끝살 너무 부드러워요 ! ...,None,"'음식이 맛있어요', '고기 질이 좋아요', '매장이 청결해요', '직접 잘 구워줘요'",23.7.1,1,
4793,sodadada7,179,0,탑골프아카데미,골프연습장,서울특별시 송파구 잠실동,"3개월 등록해서 레슨 30분으로 10번 들었어요, 일주일만에 초고속으로 배워서 라운...",None,"'시설이 깔끔해요', '타석이 넓어요', '최신 장비를 사용해요', '친절해요', ...",23.6.17,1,
4794,sodadada7,179,0,제주노을 참숯불집,한식,서울특별시 송파구 방이동,다른 거 다 필요없고 일단 고기가 진짜 맛있습니다,None,"'친절해요', '음식이 맛있어요'",23.6.5,1,


In [93]:
user_profile_df

,아이디,리뷰,팔로잉,팔로워,주소
0,sodadada7,179,1,0,https://m.place.naver.com/my/5e27e4ec8f87a842b...


저장하기

In [16]:
# 유저의 리뷰
user_df.to_excel('user_df.xlsx', index=False)

# 유저의 정보 
user_profile_df.to_excel('user_profile_df.xlsx', index=False)

# ---------------------------< test >--------------------------

In [67]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()

for url in user_list : 

    try : 
        # User의 아이디, 리뷰수, 팔로워를 가져오는 함수 
        user_data = user_profile(url)
        user_data_df = pd.DataFrame([user_data], index=[0])

        # 만약 User의 DataFrame이 이미 존재한다면, 기존 DataFrame에 추가
        if user_profile_df.index[-1] > 0:
            user_profile_df = pd.concat([user_profile_df, user_data_df], ignore_index=True)
            user_profile_df = user_profile_df.drop_duplicates() # 중복제거
        else:
            user_profile_df = user_data_df

        # 맨 처음 게시물 클릭 
        button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
        button.click()
        time.sleep(5)


        # html = driver.page_source
        # soup = BeautifulSoup(driver.page_source, 'html.parser')

        # data를 담을 빈 리스트 선언
        data_list = [] 



        # 페이지 아래로 스크롤 몇번  -> 이러면 같은거 크롤링 6번됨 ㅋㅋㅋㅋ
        page_down = 3

        # 페이지 스크롤 횟수만큼 반복
        for _ in range(page_down):
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 


        # # 반복해서 스크롤하기
        # n = 0 

        # while n < 5 :

        #     last_scroll_position = driver.execute_script("return window.scrollY")
        #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #     time.sleep(2)
        #     current_scroll_position = driver.execute_script("return window.scrollY")
            
        #     # 스크롤 위치에 변화가 없으면 종료
        #     if current_scroll_position == last_scroll_position:
        #         break
            
        #     # 현재 스크롤 위치를 다음 비교를 위해 저장
        #     last_scroll_position = current_scroll_position

        #     n += 1


            html = driver.page_source
            soup = BeautifulSoup(driver.page_source, 'html.parser')


            # 리뷰에서 정보 가져오기
            for reviews_elements in user_review_elements:
                i = 0 # 카테고리와 주소 구분할 때 사용
                restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
                
                if bool(restaurant_elements): # 정보가 없는 경우가 있음

                    # 매장명 찾기
                    restaurant_name = restaurant_elements[0].text  

            #<--------------------------------------------------------------------------------------------------->
                    # 카테고리와 주소 찾기 
                    category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
                    category_location_soup = BeautifulSoup(category_location_elements[i].get_attribute('outerHTML'), 'html.parser')
                    
                    # span 태그 안에 있는 텍스트 가져오기
                    span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
                    category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
                    location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
                    i += 1
            #<--------------------------------------------------------------------------------------------------->
                    # 리뷰 찾기 
                    review_text = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')[0].text # 리뷰 본문
                    sub_info=reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') #세부정보
                    if bool(sub_info):
                        sub_info=sub_info[0].text
                    else:
                        sub_info=None

            #<--------------------------------------------------------------------------------------------------->
                    # # 태그 찾기 
                    # # 일정 개수가 넘어가면 리뷰가 숨겨져 표시되므로 
                    # reactions_elements=reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트
                    # command=False
                    # for x in reactions_elements:
                    #     if x.get_attribute("role"): # 리액션 숨김 없애기
                    #         x.click()
                    #         command=True
                    #     else:
                    #         continue
                    # if command:
                    #     reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트 재탐색
                    # reactions=list(map(lambda x:x.text,reactions_elements))

                    # # 태그 찾기 
                    # reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b')  # 리엑션 리스트
                    # command = False

                    # for x in reactions_elements:
                    #     if x.get_attribute("role"):  # 리액션 숨김 없애기
                    #         try:
                    #             # 클릭 대기
                    #             wait = WebDriverWait(driver, 10)
                    #             wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'COw42b'))).click()
                    #             command = True
                    #         except:
                    #             # 클릭 대기 중 예외 발생 시
                    #             continue
                    # else:
                    #     if command:
                    #         # 리액션 리스트 재탐색
                    #         reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b')  
                    #     reactions = list(map(lambda x: x.text, reactions_elements))


            #<--------------------------------------------------------------------------------------------------->
                    # # 방문 일자, 재방문 이력 찾기
                    # date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
                    # date=[x.text for x in date_info]

                    # if date:
                    #     day = date[0].split('\n')[0]
                    #     if day[-1:].isdigit(): # 요일이 없으면 
                    #         weekday = day[-1:]
                    #     else:
                    #         weekday = ""
                    #     day = day[:-2]
                    #     revisit = int(date[1].replace('번째 방문', ''))

                
                    # 정보 추가
                    data_dict = {
                            '아이디' : user_data['아이디'],
                            '리뷰' : user_data['리뷰'],
                            '팔로워' : user_data['팔로워'],
                            '매장명': restaurant_name,
                            '카테고리': category,
                            '주소' : location,
                            '리뷰 내용': review_text,
                            "세부정보" : sub_info,
                            # '태그': reactions,
                            # '방문일자': day,
                            # '방문요일' : weekday,
                            # '재방문횟수': revisit
                            }
                    data_list.append(data_dict)
            user_df = pd.DataFrame(data_list)

            
            # if 'user_df' in locals():
            #     user_df = pd.concat([user_df, pd.DataFrame(data_list)], ignore_index=True)
            # else:
            #     user_df = pd.DataFrame(data_list)


    except Exception as e:
        print(f"에러 메시지: {str(e)}")
        continue
                
user_df


{'아이디': '초코피스타치오', '리뷰': 159, '팔로잉': 6, '팔로워': 24, '주소': 'https://m.place.naver.com/my/6010e880e71246c530be8c27/review?v=2'}
에러 메시지: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.161)

{'아이디': 'sodadada7', '리뷰': 179, '팔로잉': 1, '팔로워': 0, '주소': 'https://m.place.naver.com/my/5e27e4ec8f87a842bcb3505c/review?v=2'}
에러 메시지: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.161)



""


In [66]:
user_df

""


In [65]:
user_profile_df

,아이디,리뷰,팔로잉,팔로워,주소
0,sodadada7,179,1,0,https://m.place.naver.com/my/5e27e4ec8f87a842b...


In [ ]:
# 유저의 리뷰
user_df.to_excel('user_df.xlsx', index=False)

# 유저의 정보 
user_profile_df.to_excel('user_profile_df.xlsx', index=False)